# Model-View-ViewModel (MVVM) - Podstawy - Java

W tej aplikacji przyjrzymy się zastosowaniu architektury **MVVM**.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />

**Model-View-ViewModel (MVVM)** to wzorzec architektoniczny stosowany w aplikacjach mobilnych (i nie tylko), który ma na celu oddzielenie logiki biznesowej od warstwy prezentacji.

W MVVM występują trzy główne komponenty:
- **Model** - Reprezentuje dane i logikę biznesową aplikacji. Model może zawierać struktury danych, metody dostępu do bazy danych, usługi sieciowe itp.
- **View** - Odpowiada za prezentację interfejsu użytkownika (UI). Może to być np. widok, przyciski, pola tekstowe, komponenty, ekrany itp. 
- **ViewModel** - Pośredniczy między Modelem a Widokiem. ViewModel zawiera dane, które mają być wyświetlane w widoku oraz metody, które obsługują interakcje użytkownika. ViewModel nie jest bezpośrednio zależny od Widoku - jest odizolowany od konkretnego interfejsu użytkownika.

Kilka korzyści wynikających z zastosowania wzorca MVVM:
- **Separacja logiki biznesowej od warstwy prezentacji** - Dzięki MVVM można utrzymać czytelność i organizację kodu, ponieważ logika biznesowa jest oddzielona od kodu odpowiedzialnego za wygląd i interakcję z użytkownikiem.
- **Testowalność** - Dzięki oddzieleniu ViewModelu od Widoku można łatwiej testować logikę biznesową, ponieważ ViewModel **nie jest bezpośrednio zależny** od interfejsu użytkownika. Można napisać testy jednostkowe dla ViewModelu, symulując różne scenariusze bez potrzeby uruchamiania całej aplikacji.
- **Ponowne wykorzystanie kodu** Dzięki MVVM można łatwiej ponownie wykorzystywać ViewModel w różnych miejscach aplikacji. Na przykład, ten sam ViewModel może być używany w różnych ekranach, które mają podobne wymagania, ale różnią się układem i wyglądem.
- **Łatwiejsze zarządzanie stanem** MVVM pomaga w zarządzaniu stanem aplikacji poprzez wprowadzenie jednokierunkowego przepływu danych. ViewModel dostarcza dane do Widoku, które je wyświetla, a wszelkie zmiany wprowadzane przez użytkownika są przekazywane z powrotem do ViewModelu.
- **Szybsze budowanie aplikacji** - Elementy MVVM są niezależne. Zmiany w pojedynczym elemencie wymagają preezebudowania tylko tego elementu, przykładowo, zmiana w klasie `ViewModel` będzie wymagała przekompilowania tylko tej klasy, a nie całego projektu.
- **Jednokierunkowy przepływ danych** - *Unidirectional data flow* (UDF) to wzorzec projektowy, w którym stan przepływa w dół, a zdarzenia przepływają w górę. Poprzez stosowanie jednokierunkowego przepływu danych, można odseparować komponenty odpowiedzialne za wyświetlanie stanu w interfejsie użytkownika od części aplikacji, które przechowują i zmieniają stan.

<img src="https://manuelvivo.dev/assets/images/2022-06-01-viewmodel-events-antipatterns-1.png" width="200" />

Wszystkie elementy architektury znamy z poprzednich przykładów, teraz napiszemy prostą aplikację wyświetlającą imiona i nazwiska użytkowników, ktorzy będą przechowywani w liście. Wykorzystamy również *dummy data* do zainicjowania listy.

### Model

Rozpocznijmy od modelu.

In [ ]:
public class User {
    private String firstName;
    private String lastName;

    public User(String firstName, String lastName) {
        this.firstName = firstName;
        this.lastName = lastName;
    }

    public String getFirstName() {
        return firstName;
    }

    public void setFirstName(String firstName) {
        this.firstName = firstName;
    }

    public String getLastName() {
        return lastName;
    }

    public void setLastName(String lastName) {
        this.lastName = lastName;
    }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        User user = (User) o;
        return Objects.equals(firstName, user.firstName) &&
                Objects.equals(lastName, user.lastName);
    }

    @Override
    public int hashCode() {
        return Objects.hash(firstName, lastName);
    }
}


Mamy klasę `User` z dwoma polami reprezentującymi imię i nazwisko. Dodajmy `DataProvider` z danymi inicjującymi.

In [ ]:
public final class DataProvider {

    private DataProvider(){}
    private static final List<String> firstNames = Arrays.asList(
            "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
            "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    );

    private static final List<String> lastNames = Arrays.asList(
            "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
            "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
            "Nowakowski", "Pawłowski"
    );

    private static final Random random = new Random();

    public static List<User> getUsers() {
        return generateUsers(41);
    }

    private static List<User> generateUsers(int count) {
        List<User> users = new ArrayList<>();
        for (int i = 0; i < count; i++) {
            String firstName = getRandomElement(firstNames);
            String lastName = getRandomElement(lastNames);
            User user = new User(firstName, lastName);
            users.add(user);
        }
        return users;
    }

    private static String getRandomElement(List<String> list) {
        int index = random.nextInt(list.size());
        return list.get(index);
    }
}

### ViewModel

W klasie `UserViewModel` przechowujemy listę wszystkich użytkowników jako `MutableLiveData<List<User>>`.

Tutaj napotykamy pewien problem, `MutableLiveData` przechowuje i daje możliwość obserwacji samej listy `List<User>`, ale nie jej elementów. Więc dodając nowy element chcemy utrworzyć nową listę i zastąpić tą przechowywaną w zmiennej `userList`.

In [ ]:
public class UserViewModel extends ViewModel {
    private final MutableLiveData<List<User>> usersList = new MutableLiveData<>();
    public LiveData<List<User>> getUsersList() {
        return usersList;
    }

    public UserViewModel() {
        reinitialize();
    }

    public void addUser(User user) {
        if (usersList.getValue() != null) {
            List<User> currentList = new ArrayList<>(usersList.getValue());
            currentList.add(user);
            currentList.sort(Comparator.comparing(User::getLastName));
            usersList.setValue(currentList);
        }
    }

    public void reinitialize() {
        List<User> sortedUsers = new ArrayList<>(DataProvider.getUsers());
        sortedUsers.sort((user1, user2) -> {
            if (user1.equals(user2)) { return 0; }
            int result = user1.getLastName().compareTo(user2.getLastName());
            if (result == 0) {
                return user1.getFirstName().compareTo(user2.getFirstName());
            }
            return result;
        });
        usersList.setValue(sortedUsers);
    }

    public void clear() {
        usersList.setValue(Collections.emptyList());
    }
}


- `private final MutableLiveData<List<User>> usersList = new MutableLiveData<>();` - lista użytkowników, do której można uzyskać dostęp tylko do odczytu.
- `public void addUser(User user)` - Metoda służy do dodawania nowego użytkownika do listy. 
- `public void reinitialize()` - Metoda służy do zresetowania zawartości listy użytkowników do wartości domyślnych.
- `public void clear()` - Metoda służy do wyczyszczenia całej listy użytkowników.

### View

Dodajmy `View` do aplikacji, czyli zdefiniujmy interfejs użytkownika. Dodajmy layouty dla aktywności, fragmentu oraz pojedynczego elementu listy.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.MainActivity">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/fragment_main"
        android:name="com.example.mvvmbasicskotlin.ui.fragment.UsersFragment"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="First Name">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/firstNameEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <LinearLayout
            android:layout_width="0dp"
            android:layout_weight="1"
            android:layout_height="wrap_content"
            android:layout_margin="4dp"
            android:orientation="horizontal">

            <com.google.android.material.textfield.TextInputLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="Last Name">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/lastNameEditText"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:fontFamily="sans-serif"
                    android:textSize="15sp" />
            </com.google.android.material.textfield.TextInputLayout>
        </LinearLayout>

        <Button
            android:id="@+id/addButton"
            android:layout_width="wrap_content"
            android:layout_gravity="center"
            android:text="ADD"
            android:layout_margin="4dp"
            android:layout_height="wrap_content"/>
    </LinearLayout>

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/rvList"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_margin="4dp"/>

    <Button
        android:id="@+id/clearButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginStart="8dp"
        android:layout_marginEnd="8dp"
        android:text="CLEAR"
        android:layout_height="wrap_content"/>

    <Button
        android:id="@+id/resetButton"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_marginStart="8dp"
        android:layout_marginEnd="8dp"
        android:layout_marginBottom="8dp"
        android:text="RESET"
        android:layout_height="wrap_content"/>

</LinearLayout>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="horizontal">

    <TextView
        android:id="@+id/wordTextView"
        android:layout_width="0dp"
        android:layout_weight="1"
        android:layout_height="wrap_content"
        android:layout_margin="4dp"
        android:text="slowo"
        android:textAlignment="center"
        android:layout_marginStart="8dp"
        android:textSize="24sp" />

</LinearLayout>

Zaimplementujmy elementy `RecyclerView`

In [ ]:
public class UserViewHolder extends RecyclerView.ViewHolder {
    private RvItemBinding binding;

    public UserViewHolder(RvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(User item) {
        String fullName = item.getFirstName() + " " + item.getLastName();
        binding.wordTextView.setText(fullName);
    }
}

In [ ]:
public class UserComparator extends DiffUtil.ItemCallback<User> {
    @Override
    public boolean areItemsTheSame(@NonNull User oldItem, @NonNull User newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(User oldItem, @NonNull User newItem) {
        return oldItem.equals(newItem);
    }
}

In [ ]:
public class UserAdapter extends ListAdapter<User, UserViewHolder> {
    public UserAdapter(@NonNull DiffUtil.ItemCallback<User> diffCallback) {
        super(diffCallback);
    }

    @NonNull
    @Override
    public UserViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        LayoutInflater inflater = LayoutInflater.from(parent.getContext());
        RvItemBinding binding = RvItemBinding.inflate(inflater, parent, false);
        return new UserViewHolder(binding);
    }

    @Override
    public void onBindViewHolder(@NonNull UserViewHolder holder, int position) {
        User item = getItem(position);
        holder.bind(item);
    }
}

Następnie przejdźmy do `UsersFraggment`

Potrzebujemy dwie wartości przechowujące instancję `UserViewModel` oraz `UserAdapter`

In [ ]:
private UserViewModel viewModel;
private UserAdapter userAdapter;

Zainicjujmy je w metodzie `OnCreateView`

In [ ]:
viewModel = new ViewModelProvider(this).get(UserViewModel.class);
userAdapter = new UserAdapter(new UserComparator());

Dodajmy fragment jako obserwator, oraz podepnijmy odpowiednie metody do elementów layoutu.

In [ ]:
@Override
public View onCreateView(LayoutInflater inflater, ViewGroup container, Bundle savedInstanceState) {
    binding = FragmentUsersBinding.inflate(inflater, container, false);

    viewModel = new ViewModelProvider(this).get(UserViewModel.class);
    userAdapter = new UserAdapter(new UserComparator());

    viewModel.getUsersList().observe(getViewLifecycleOwner(), users -> userAdapter.submitList(new ArrayList<>(users)));

    binding.rvList.setAdapter(userAdapter);
    binding.rvList.setLayoutManager(new LinearLayoutManager(requireContext()));

    binding.addButton.setOnClickListener(v -> onAddUser());
    binding.resetButton.setOnClickListener(v -> onResetUsers());
    binding.clearButton.setOnClickListener(v -> onClearUsers());

    return binding.getRoot();
}

`viewModel.getUsersList().observe(getViewLifecycleOwner(), users -> ...);` - Ta linia kodu używa metody `observe` na polu `usersList` z klasy `UserViewModel`. Umożliwia ona reagowanie na zmiany w liście użytkowników. `viewLifecycleOwner` wskazuje, że obserwator będzie aktywny w cyklu życia fragmentu. W bloku lambdy ` users -> ... ` następuje reakcja na zmiany w liście - przekazuje ona nową listę użytkowników do `userAdapter`.

Następnie dodajmy metody obsługujące dodanie nowego użytkownika, czyszczenie i reinicjalizację danych.

In [ ]:
private void onAddUser() {
    if (binding.firstNameEditText.getText() != null && binding.lastNameEditText.getText() != null) {
        String firstName = binding.firstNameEditText.getText().toString().trim();
        String lastName = binding.lastNameEditText.getText().toString().trim();

        if (!firstName.isEmpty() && !lastName.isEmpty()) { // sprawdzam czy pola nie są puste
            User newUser = new User(firstName, lastName);  // tworzę nowego użytkownika
            viewModel.addUser(newUser);                    // dodaję użytkownika do listy

            binding.firstNameEditText.getText().clear();   // czyszczę listę
            binding.lastNameEditText.getText().clear();
        }
    }
}

private void onResetUsers() {
    viewModel.reinitialize();
}

private void onClearUsers() {
    viewModel.clear();
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />